<img height="60px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="20px" vspace="5px">

# Solution 1 - Simple

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/Kaggle-State-Farm-Distracted-Driver-Detection/Solution%201-%20Simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/Kaggle-State-Farm-Distracted-Driver-Detection/Solution%201-%20Simple.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Import libraries/ dependencies

In [0]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob
import random
import cv2
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt     
from keras.preprocessing import image                  
from tqdm import tqdm
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import log_loss
import math
import pickle
import os

Import images

In [0]:

def load_train (path):
    data=load_files(path)
    driver_file=np.array(data['filenames'])
    driver_target=np_utils.to_categorical(np.array(data['target']),10)
    
    return driver_file,driver_target

def load_test(path):
    data=glob(path)
    driver_file=np.array(glob(path))
    driver_id=[]
    for p in data:
        driver_id.append(os.path.basename(p))
   # driver_id=os.path.basename(p)
    #test_id=np.array(driver_file['target'])
    return driver_file,driver_id

#submission training set
train_files,train_targets = load_train('Data/imgs/train')

#Submission Test set - has no labels
test_files,test_id = load_test('Data/imgs/test/*')


Split Data into train and test

In [0]:
# My own test set - split the training set 
X_train, X_test, y_train, y_test = train_test_split(train_files,train_targets ,test_size=0.3,random_state=42)

#optional: split the test set to get a validation set
    #X_train, X_test, y_train, y_test = train_test_split(train_targets,train_files, test_size=0.3,random_state=42)

Data Analysis

In [0]:
#Final datasets
print('training targets size %d' % len(train_targets))
print('test targets size %d' % len(test_files))

#first 5 sample values
for q in train_files[:5]: print(q)
for p in train_targets[:5]: print (p)

    
    
#My own dataset
print('train size %d' % len(X_train))
print('test size %d' % len(X_test))

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)



path=X_train[1]
print(path)
im = cv2.imread(path)
#cv2.imshow(im)
plt.imshow(im)
plt.show()
#img=cv2.imread(X_train[1])
#cv2.imshow('img',img)
#plt.imshow(img)
#plt.show()
for p in X_train[:5]: print (p)
for p in y_train[:5]: print (p)

for p in X_test[:5]: print (p)
for p in y_test[:5]: print (p)

Pre-process data

In [0]:


def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)                

# pre-process the data for Keras
train_targets=y_train
test_targets=y_test
train_tensors = paths_to_tensor(X_train).astype('float32')/255
test_tensors = paths_to_tensor(X_test).astype('float32')/255


Image Enhancements

Model Architecture

In [0]:

model = Sequential()
### TODO: Define your architecture.
print('x_train shape:', train_tensors.shape)
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))


model.summary()

Model Compilation

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Model Training

In [0]:
checkpointer = ModelCheckpoint(filepath='saved_models/weightsfromscratch.hdf5', 
                               verbose=1, save_best_only=True)

In [0]:
history=model.fit(train_tensors, train_targets, validation_split=0.2,epochs=5, batch_size=20, callbacks=[checkpointer], verbose=1)

Model accuracy etc

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Model Testing

In [0]:
model.load_weights('saved_models/weightsfromscratch.hdf5')
#score = model.evaluate(test_tensors, test_targets, verbose=1)
#print(score)

Model results

In [0]:
print('helloworld')
predictions=model.predict(test_tensors, verbose=1)
score=log_loss(test_targets,predictions)
print('Score log_loss: ', score)

In [0]:
def mlogloss(target, pred):
    score = 0.0
    for i in range(len(pred)):
        pp = pred[i]
        for j in range(len(pp)):
            prob = pp[j]
            if prob < 1e-15:
                prob = 1e-15
            score += target[i][j] * math.log(prob)
    return -score/len(pred)
score = mlogloss(test_targets, predictions)
print('Score : mlogloss', score)

**Kaggle Submission**

Reduce test set size

In [0]:
test_files=test_files[:10000]
test_id=test_id[:10000]
final_test_tensors = paths_to_tensor(test_files).astype('float32')/255

Predict complete test set

In [0]:
predictions=model.predict(final_test_tensors, verbose=1)

Create submission format

In [0]:
def create_submission(predictions, test_id, loss):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = str(round(loss, 6)) + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)

In [0]:
create_submission(predictions,test_id,score)